# Introduction to Regularization

<a href="https://drive.google.com/file/d/1EZ_xqMaYj77vErVnrQmnFOj-VBEoO5uW/view" target="_blank">
     <img src="http://www.deltanalytics.org/uploads/2/6/1/4/26140521/screen-shot-2019-01-05-at-4-48-29-pm_orig.png" width="500" height="400">
</a>

In the context of regression, regularization refers to techniques to constrain/shrink the coefficient estimates towards zero.  
Shrinking the coefficients can 1) improve the fit of the model and 2) reduce the variance of the coefficients.

Two common types of regularization are ridge and lasso.  

Recall that least squares linear regression minimizes the residual sum of squares (RSS).  In other words, it minimizes

$ RSS = \displaystyle \sum^{n}_{i=1} (y_i - \beta_0 - \sum^{p}_{j=1} \beta_j x_{ij})^2 $

In ridge and lasso, we add a term to the value we are trying to minimize.  

In ridge, we minimize 

$ RSS + \lambda \displaystyle \sum^{p}_{j=1} \beta_j^2 $

In lasso, we minimize

$ RSS + \lambda \displaystyle \sum^{p}_{j=1} |\beta_j| $

The $\lambda$ (pronounced "lambda") in the above values is a hyper-parameter which determines how 'strong' the regularization effect is.  Note: sometimes $\alpha$ (pronounced "alpha") is used instead of $\lambda$.

A useful way to use ridge or lasso regression is to run the regression over a range of alphas and see which features maintain a large beta coefficient for the longest. It is these features which have the most predictive power!

More in depth information can be found here: [Regularization Regression](https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/)

In [ ]:
# Load python packages
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
plt.rcParams['figure.figsize'] = (12, 8)
sns.set()
sns.set(font_scale=1.5)

# packages for checking assumptions
from scipy import stats as stats
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
import statsmodels.formula.api as sm

# packages for regularization
from sklearn.linear_model import Lasso
from math import pow, sqrt

np.random.seed(1234)

2) Load Data
---

In [ ]:
# Load data
path = '../data/'
filename = 'loans.csv'
df = pd.read_csv(path+filename)

In [ ]:
# Alternatively, if you are using Colab, get the data by git cloning the Delta Analytics repository
!git clone https://github.com/DeltaAnalytics/machine_learning_for_good_data
df = pd.read_csv("machine_learning_for_good_data/loans.csv")

In [ ]:
df.dtypes

In [ ]:
# create indicator variables for country
for country in df['location_country_code'].unique():
    if country is not np.nan:
        df['country_'+country] = np.where(df.location_country_code == country, 1, 0)

In [ ]:
# create indicator variables for sector
for sect in df['sector'].unique():
    df['sector_'+sect] = np.where(df.sector == sect, 1, 0)

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

# Define the dependent variable
y = df['loan_amount']
# Define the independent variables
X = df[['lender_count', 'sector_Education', 'sector_Clothing', 
        'sector_Personal Use', 'sector_Retail', 'sector_Transportation', 'sector_Agriculture']]
# Add an intercept term to the independent variables
X['cnst'] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model1 = sm.OLS(endog = y_train,exog = X_train).fit()
print(model1.summary())

In [ ]:
alphas = np.arange(0.001, 0.502, 0.002)
lasso_coefs = []
X_train_lasso= X_train[X_train.columns.tolist()] # Select columns / features for model

for a in alphas:
    lassoreg = Lasso(alpha=a, copy_X=True, normalize=True)
    lassoreg.fit(X_train_lasso, y_train)
    lasso_coefs.append(lassoreg.coef_)

In [ ]:
lasso_coefs = np.asarray(lasso_coefs).T

plt.figure(figsize=(14,10))
for coefs, feature in zip(lasso_coefs, X_train_lasso.columns):
    plt.plot(alphas, coefs, label = feature)
plt.legend(loc='best')
plt.show()

Retail and Transportation go to 0 when alpha is 0.3.  Let's try removing these from the model.

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

# Define the dependent variable
y = df['loan_amount']
# Define the independent variables
X = df[['lender_count', 'sector_Education', 'sector_Clothing', 
        'sector_Personal Use', 'sector_Agriculture']]
# Add an intercept term to the independent variables
X['cnst'] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model2 = sm.OLS(endog = y_train,exog = X_train).fit()
print(model2.summary())

Even though we removed two dependent variables from the analysis, our R-squared and adjusted R-squared stayed the same.  This means that the two variables we removed (Transportation and Retail) are less important to loan amount.  The example above shows how we can use regularization for feature selection.

# Important facts about regularization

Recall that with least squares linear regression, the coefficients are scale equivariant.  In other words, multiplying a feature by a constant $c$ simply leads to a scaling of the least squares coefficient estimate by a factor of 1/$c$. 

Let's demonstrate this fact by creating a example set of data that has three variables: 1) amount of money made at a restaurant in one day, 2) distance in meters to the nearest university, 3) distance in kilometers to the nearest hospital.

In [ ]:
np.random.seed(1234)

earnings = np.random.normal(2000, 300, 50)
university_distances = np.random.normal(7000,2000,50)
hospital_distances = np.random.normal(7,2,50)

earnings = [a if a > 0 else -a for a in earnings]
university_distances = [a if a > 0 else -a for a in university_distances]
hospital_distances = [a if a > 0 else -a for a in hospital_distances]

In [ ]:
df = pd.DataFrame({"earnings": sorted(earnings), "university": sorted(university_distances, reverse=True), 
                   'hospital' : sorted(hospital_distances, reverse=True)})

In [ ]:
df

In [ ]:
# plot distance to nearest university (in meters) vs. earnings
ax = sns.regplot(x='earnings', y='university', data=df, fit_reg=False)
ax.set_title('Scatter plot of distance to nearest university (in meters) vs earnings')

In [ ]:
# plot distance to nearest hospital (in kilometers) vs. earnings
ax = sns.regplot(x='earnings', y='hospital', data=df, fit_reg=False)
ax.set_title('Scatter plot of distance to nearest hospital (in kilometers) vs earnings')

Let's run a multivariate linear regression without scaling any variables and compare the results to a model where we standardize the distance variables to both use kilometers.

In [ ]:
model1 = sm.ols(formula = 'earnings ~ university + hospital', data = df).fit()
print(model1.summary())

The R-squared is 0.938 and the Adjusted R-squared is 0.935.  The coefficients for the intercept, university, and hospital are 3024.1009, -0.0643, and -76.3083.  Now let's scale the university variable to be in kilometers instead of meters.

In [ ]:
df_scaled = df.copy()
df_scaled['university'] = df_scaled['university']/1000

df_scaled

In [ ]:
model2 = sm.ols(formula = 'earnings ~ university + hospital', data = df_scaled).fit()
print(model2.summary())

The R-squared is 0.938 and the Adjusted R-squared is 0.935. The coefficients for the intercept, university, and hospital are 3024.1009, -64.3473, and -76.3083.   So we changed the university variable by scaling it by a constant and the resulting coefficient was scaled by the same constant.  The p-values did not change and the coefficients on the other variables did not change.

What do you think scaling will do if we incorporate regularization by using lasso or ridge regression?  Do you think scaling will have an effect on the coefficients of the variables?

<br>
<br>
<br>
<br>


Let's run lasso on our unscaled data and our scaled data and see what happens.

# Unscaled data

In [ ]:
X = df[['university', 'hospital']]
y = df['earnings']

alphas = np.arange(0.001, 1, 0.002)
lasso_coefs = []
X_lasso= X[X.columns.tolist()] # Select columns / features for model

for a in alphas:
    lassoreg = Lasso(alpha=a, copy_X=True, normalize=True)
    lassoreg.fit(X_lasso, y)
    lasso_coefs.append(lassoreg.coef_)
    
lasso_coefs = np.asarray(lasso_coefs).T

plt.figure(figsize=(14,10))
for coefs, feature in zip(lasso_coefs, X_lasso.columns):
    plt.plot(alphas, coefs, label = feature)
plt.legend(loc='best')
plt.show()

The above plot shows the coefficients for the university and hospital variables at 0 and approximately -75, respectively.  Would you keep or drop these variables from your model?  Why?

<br>
<br>
<br>
<br>

# Scaled data

In [ ]:
X = df_scaled[['university', 'hospital']]
y = df_scaled['earnings']

alphas = np.arange(0.001, 1, 0.002)
lasso_coefs = []
X_lasso= X[X.columns.tolist()] # Select columns / features for model

for a in alphas:
    lassoreg = Lasso(alpha=a, copy_X=True, normalize=True)
    lassoreg.fit(X_lasso, y)
    lasso_coefs.append(lassoreg.coef_)
    
lasso_coefs = np.asarray(lasso_coefs).T

plt.figure(figsize=(14,10))
for coefs, feature in zip(lasso_coefs, X_lasso.columns):
    plt.plot(alphas, coefs, label = feature)
plt.legend(loc='best')
plt.show()

The above plot shows the coefficient for the university and hospital variables are at around -64 and -76, respectively.  Would you keep or drop these variables from your model?  Why?

<br>
<br>
<br>
<br>

Clearly, scaling affects the coefficients and thus affects the results of lasso regression.  Thus, it is best to apply regularization techniques like ridge and lasso after standardizing the predictors.  You can standardize the predictors by applying the following formula:

$ \tilde{x}_{ij} = \frac{x_{ij}}{\sqrt{\frac{1}{n} \sum_{i=1}^{n} (x_{ij} - \bar{x}_{j})^2}} $

So now let's take the unscaled data and make a new dataset where we standardize the predictors.